# Data processing

In [38]:
import pandas as pd 
import glob 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import os 
import sklearn
import scipy.sparse
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype
os.getcwd()

'/Users/julesbaudet/Documents/0. Cours/ENS/Deep Learning DIY/Projet final/DL_ens'

In [39]:
path = "/Users/julesbaudet/Documents/0. Cours/ENS/Deep Learning DIY/Projet final/ens_data"
# path = "/Users/linusbleistein/Documents/Cours ENS/Cours mathématiques/Deep learning 2020-2021/data_project"

In [40]:
df = pd.read_csv(os.path.join(path,"clean_brave_data.csv")).sort_values('unix_block_timestamp')
df.shape

(2695566, 9)

In [41]:
df = df.drop(columns = 'Unnamed: 0')

## Transaction History Embedding

In [42]:
# import the known exchange list 

exch_labels = pd.read_csv(os.path.join(path, "exchanges_encoded.csv"), 
                          delimiter=';', names  = ['address','label']).set_index('address')

labels = exch_labels.to_dict()['label']

labels2int = dict(zip(labels.values(),[i for i in range(len(labels))])) # keys = addresses, values = name of the exchange
address2int = {k:labels2int[labels[k]] for k in labels.keys()}

known_addresses = list(labels.keys())

In [43]:
# drop transactions of addresses who only made 1 transaction 

a = (df.groupby('from_address').count() + df.groupby('to_address').count())['value']
idx = a[a<3][~a[a<3].index.isin(known_addresses)].index
df = df.drop(df.loc[df['from_address'].isin(idx)|df['from_address'].isin(idx)].index).reset_index()

In [44]:
df['transaction_id'] = df['from_address'] + df['to_address']
df = df.drop(columns = ['labels_send', 'labels_receive'])
# np.savetxt('data/trainset.txt',df['transaction_id'].values, fmt = '%s', delimiter =' ', newline =' ')

In [45]:
%%time
# add information on transactions
df['tfrequency'] = df.groupby('transaction_id')['value'].transform('count') 
# df['avg_value'] = df.groupby('transaction_id')['value'].transform('mean') 
df['std_value'] = df.groupby('transaction_id')['value'].transform('std') 
df['median_value'] = df.groupby('transaction_id')['value'].transform('median') 
df['lifetime'] = df.groupby('transaction_id')['unix_block_timestamp'].transform('max') - df.groupby('transaction_id')['unix_block_timestamp'].transform('min')
df['lifetime'] = df['lifetime']/df['lifetime'].max()
df['first_date'] = df.groupby('transaction_id')['unix_block_timestamp'].transform('min')
df['unix_block_timestamp'] = (df['unix_block_timestamp']-df['unix_block_timestamp'].min())/(df['unix_block_timestamp'].max()-df['unix_block_timestamp'].min())


data = df.groupby('transaction_id').mean()

CPU times: user 33.5 s, sys: 1.47 s, total: 34.9 s
Wall time: 44.7 s


In [46]:
data.columns

Index(['index', 'value', 'unix_block_timestamp', 'send_is_id', 'receive_is_id',
       'tfrequency', 'std_value', 'median_value', 'lifetime', 'first_date'],
      dtype='object')

In [47]:
# import shutil
# 
# source_file = open('data/final_output.txt', 'r')
# source_file.readline()
# # this will truncate the file, so need to use a different file name:
# target_file = open('final_output2.txt.new', 'w')
# 
# shutil.copyfileobj(source_file, target_file)

In [48]:
emb = pd.read_csv('data/model.txt', skiprows=1, sep=' ', header = None, index_col = 0)

# Comment that line when you have full embedding 
#data = data.loc[data.index.isin(emb.index)] 
#emb = emb.loc[emb.index.isin(data.index)] 
#############

In [49]:
emb.shape

(945377, 150)

In [50]:
trans_data = pd.concat((emb,data),axis=1)

trans_data[['from_address','to_address']] = trans_data.index.str.extract('(.{42,42})' * 2).values

In [51]:
dta1 = trans_data.sort_values('from_address').reset_index().rename(columns = {'from_address':'address', 'send_is_id':'label'})
dta1 = dta1.drop(columns = ['to_address','receive_is_id'])
dta2 = trans_data.loc[trans_data['from_address'] != trans_data['to_address']].sort_values('to_address').reset_index()
dta2 = dta2.rename(columns = {'to_address':'address', 'receive_is_id':'label'}).drop(columns = ['from_address','send_is_id'])

##########
# dta2 = dta1.copy() # TO DELETE LATER 
##########

dta2['value'] = - dta2['value']
dta2['median_value'] = - dta2['median_value']

In [52]:
final_data = pd.concat((dta1,dta2)).sort_values(['address','first_date']).drop(columns=['index','first_date'])

In [53]:
final_data

,level_0,1,2,3,4,5,6,7,8,9,...,149,150,value,unix_block_timestamp,label,tfrequency,std_value,median_value,lifetime,address
0,0x3e6288bc299dac35d293949e08faae61547d297d0x00...,0.002860,0.002553,0.001282,0.000694,-0.002919,-0.000901,0.000579,-0.000329,-0.000971,...,0.002145,-0.000914,-475350.000000,0.026648,0.0,1.0,NaN,-475350.000000,0.000000,0x0000000000000000000000000000000000000000
5,0xb14ce8465a0a2ec321c7da6c9313103dff762ef00x00...,-0.001349,0.001012,-0.002908,0.001011,0.000784,0.001800,0.000218,0.000885,-0.000613,...,0.001322,-0.000831,-3381.990227,0.033807,0.0,1.0,NaN,-3381.990227,0.000000,0x0000000000000000000000000000000000000000
4,0xf7c8b97a2f8e6df7e0126e6fa953e257b7de9dc20x00...,-0.000052,-0.001626,0.001550,0.000116,0.001593,-0.003225,-0.001264,-0.000218,-0.002452,...,0.001393,0.000345,-4.000000,0.628644,0.0,1.0,NaN,-4.000000,0.000000,0x0000000000000000000000000000000000000000
2,0x340d693ed55d7ba167d184ea76ea2fd092a35bdc0x00...,0.003111,0.001909,0.003098,-0.003031,-0.000591,-0.000034,-0.002775,-0.002273,-0.001554,...,0.000899,-0.000604,-12.387875,0.849646,0.0,2.0,17.342678,-12.387875,0.120074,0x0000000000000000000000000000000000000000
3,0xd8322486c4e13ad800bde0174b7397bb86189ea20x00...,-0.000961,-0.001208,-0.001371,0.001580,0.000663,0.000467,-0.002572,0.002252,-0.002978,...,0.002091,0.002941,-2.016700,0.874982,0.0,1.0,NaN,-2.016700,0.000000,0x0000000000000000000000000000000000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944426,0x35f4db7d3bc121562f99aab530b0f95330ca70710xff...,0.002740,0.000504,-0.000125,0.000434,-0.000385,0.001274,0.002561,0.003069,-0.002111,...,0.000742,0.000038,-32.404034,0.594525,0.0,1.0,NaN,-32.404034,0.000000,0xffffc583d80cf66aa156eaee3bf5f185890c02de
944427,0x340d693ed55d7ba167d184ea76ea2fd092a35bdc0xff...,0.000377,0.001894,-0.001994,0.001631,-0.000927,-0.003018,0.002701,-0.002421,0.000330,...,-0.002164,-0.000229,-50.000000,0.742843,0.0,1.0,NaN,-50.000000,0.000000,0xffffd78093141d7e2eb923b76053effcec377fbf
944428,0xe93381fb4c4f14bda253907b18fad305d799241a0xff...,-0.002737,-0.002992,-0.001259,0.002574,0.000814,-0.001486,0.002444,-0.001596,-0.000914,...,-0.002560,-0.002364,-200.000000,0.524202,0.0,1.0,NaN,-200.000000,0.000000,0xffffe661474073010cb7ab1f8af7b4505af6df5c
944429,0x794a8a2a9e7e360afd5290119c775ffa6b1af7c00xff...,-0.002909,-0.003191,-0.002699,-0.000057,0.002525,-0.001665,0.001021,0.002712,-0.001537,...,-0.001811,0.001249,-224.906038,0.880727,0.0,1.0,NaN,-224.906038,0.000000,0xfffffe85ddb377d1a616cf2c43ae5556ad3987e8


In [54]:
final_data = final_data.rename(columns={'level_0':'address_id'})

# MAYBE REINDEX???? 

In [55]:
final_data = final_data.groupby('address').mean()



In [57]:
final_data.to_csv(os.path.join(path,"grouped_data_final.csv"))

In [ ]:
#### fake dataset for debugging
emb2 = pd.read_csv('data/model_test.txt', sep=' ', header =None, index_col = 0).reset_index()
trans_data2 = data.iloc[:emb2.shape[0]].reset_index()

In [ ]:
emb2[trans_data2.columns] = trans_data2
trans_data = emb2.rename(columns={0:'address_id'})
trans_data[['from_address','to_address']] = trans_data.address_id.str.extract('(.{42,42})' * 2).values
dta1 = trans_data.sort_values('from_address').reset_index().rename(columns = {'from_address':'address', 'send_is_id':'label'})
dta1 = dta1.drop(columns = ['to_address','receive_is_id'])
dta2 = trans_data.loc[trans_data['from_address'] != trans_data['to_address']].sort_values('to_address').reset_index()
dta2 = dta2.rename(columns = {'to_address':'address', 'receive_is_id':'label'}).drop(columns = ['from_address','send_is_id'])

##########
# dta2 = dta1.copy() # TO DELETE LATER 
##########

dta2['value'] = - dta2['value']
dta2['median_value'] = - dta2['median_value']
final_data = pd.concat((dta1,dta2)).sort_values(['address','first_date']).drop(columns=['index','first_date'])

In [ ]:
final_data['label'] = np.random.choice(2 ,p=[0.9,0.1], size = 1307) 
final_data = final_data.drop(columns='level_0')
final_data.to_csv(os.path.join(path,"test_data.csv"))

# Old tentatives (do not run these cells)

##  Adding the labels

In [ ]:
labels = np.vectorize(address2int.get)(np.array(categories.categories))
labels = np.nan_to_num(labels.astype(float))

In [ ]:
labels.shape

In [ ]:
data = pd.DataFrame(out)
# data['label'] = np.isin(np.array(sorted(df.from_address.unique())),known_addresses).astype(int)
data['label'] = labels

In [ ]:
data.to_csv(os.path.join(path,"processed_data.csv"))

## Manual embedding (Old)

In [ ]:
# now we create a dictionnary address: [transaction nbrs] where the transaction nbrs involve the address
all_addresses  = np.hstack((df.from_address.unique(),
                            df.loc[~df['to_address'].isin(df['from_address'])].to_address.unique()))

In [ ]:
to_add = df.loc[~df['to_address'].isin(df['from_address'])].copy()
to_add = to_add.rename(columns={'to_address':'from_address','from_address':'to_address'})
to_add['value'] = 0
df2 = pd.concat((df,to_add))

categories = CategoricalDtype(sorted(df2.from_address.unique()), ordered=True) # will have all nodes! use it for all 

In [ ]:
from_c = df.from_address.astype(categories).cat.codes # .unique().shape
to_c = df.loc[df.from_address != df.to_address].to_address.astype(categories).cat.codes

vocab = pd.DataFrame(np.hstack((from_c,to_c)), columns=['address'])
vocab['transactions'] = np.hstack((from_c.index, to_c.index))

In [ ]:
%%time
vocab = vocab.groupby('address')['transactions'].apply(list).reset_index(name='transactions')

In [ ]:
# ARE THEY IN ORDER? CHECK! 
vocab.head()

In [ ]:
# %%time
# %run word2vec/train.py

In [ ]:
# sum([len(v)!=42 for v in df['from_address'].values]) # check that all addresses have same format

In [ ]:
# df['transaction_id'] = df['from_address']+df['to_address']

In [ ]:
# df['transaction_id'].nunique()

In [ ]:
# df = df.sort_values('unix_block_timestamp')

In [ ]:
# vocab_size = df['transaction_id'].nunique()
# word2vec = Word2Vec(vocab_size=vocab_size, embedding_size=300)
# sgns = SGNS(embedding=word2vec, vocab_size=vocab_size, n_negs=20)
# optim = Adam(sgns.parameters())
# for batch, (iword, owords) in enumerate(dataloader):
#     loss = sgns(iword, owords)
#     optim.zero_grad()
#     loss.backward()
#     optim.step()

In [ ]:
# first we create an one-hot encoding of all transactions ranked by chronological order
#hot_encoding =  csr_matrix((np.ones(df.shape[0]),(df.index.values, df.index.values)), shape=(df.shape[0], df.shape[0]))

In [ ]:
#a = '0x88e2efac3d2ef957fcd82ec201a506871ad06204'
#np.unique(np.hstack((df.from_address.values, df['to_address'].values))).shape

## Creating a data  embedding (OLD)

To be able to learn a classification of nodes on our graph, we need to transform the data on nodes to be able to express some of the information contained in graph form.

In [ ]:
# matrix of adjacency with line data relative to sent transactions only 
# %%time
# from_c = CategoricalDtype(sorted(df.from_address.unique()), ordered=True)
# to_c = CategoricalDtype(sorted(df.to_address.unique()), ordered=True)
# 
# row = df.from_address.astype(from_c).cat.codes
# col = df.to_address.astype(to_c).cat.codes
# sparse = csr_matrix((df["value"], (row, col)), \
#                            shape=(from_c.categories.size, to_c.categories.size))
# print(sparse.shape)

In [ ]:
# matrix of adjacency with line data relative to sent and received transactions 
%%time
# first we add addresses that only received but not sent to first column 
to_add = df.loc[~df['to_address'].isin(df['from_address'])].copy()
to_add = to_add.rename(columns={'to_address':'from_address','from_address':'to_address'})
to_add['value'] = 0

df = pd.concat((df,to_add))

categories = CategoricalDtype(sorted(df.from_address.unique()), ordered=True) # will have all nodes! use it for all 
row = df.to_address.astype(categories).cat.codes
col = df.from_address.astype(categories).cat.codes

# sparse matrix for values sent 
sp = csr_matrix((df['value'],(row, col)), shape=(categories.categories.size, categories.categories.size)
                
# sparse matrix for values received 
sp2 = csr_matrix((-df['value'],(col, row)), shape=(categories.categories.size, categories.categories.size))
                
# our data 
X = scipy.sparse.hstack((sp,sp2))

In [ ]:
X.shape

##  Dimension Reduction   

Our data has particularly high dimension. We use truncated SVD to reduce our data set. 

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
%%time
tsvd = TruncatedSVD(n_components=65, random_state=42)
out = tsvd.fit_transform(X)
print(f'our data has shape {out.shape}')

In [ ]:
tsvd_var_ratios = tsvd.explained_variance_ratio_
print(tsvd_var_ratios.sum())

In [ ]:
# taken on https://chrisalbon.com/machine_learning/feature_engineering/select_best_number_of_components_in_tsvd/
def select_n_components(var_ratio, goal_var: float) -> int:
    total_variance = 0.0
    n_components = 0
    for explained_variance in var_ratio:
        total_variance += explained_variance
        n_components += 1
        if total_variance >= goal_var:
            break
    return n_components

var_goal =  0.99
print(f'to keep {var_goal*100}% of variance, one should keep {select_n_components(tsvd_var_ratios,var_goal)} components')

In [ ]:
# # Debugging example
# a = pd.DataFrame({"col": [15, 32, 3, 8], "col2":[26,3,17,20], "value":[1,1,1,1]})
# 
# # add this to the sparse matrix 
# add = a.loc[~a['col2'].isin(a['col'])].copy()
# add = add.rename(columns={'col':'col2','col2':'col'})
# #add['value'] = 0 # -to_add['value']
# 
# a = pd.concat((a,add))
# 
# categories = CategoricalDtype(sorted(a.col.unique()), ordered=True) # will have all nodes! use it for all 
# row = a.col.astype(categories).cat.codes
# col = a.col2.astype(categories).cat.codes
# sp = csr_matrix((a['value'],(row, col)), shape=(categories.categories.size, categories.categories.size))

## Handling time

In [ ]:
# from datetime import datetime

In [ ]:
# df['day'] = df['unix_block_timestamp']//(60*60*24)  # euclidean division to get the day number 

In [ ]:
# # tests
# df['unix_block_timestamp'].max()//(60*60*24)
# datetime.utcfromtimestamp(df['unix_block_timestamp'].max())
# datetime.utcfromtimestamp(df['unix_block_timestamp'].min())

In [ ]:
# df.groupby(by='day').count().plot(y='value')

In [ ]:
#  plt.subplots(figsize = (20,7))
#  test = df.groupby(['day']).nunique()['from_address']
#  test.hist(bins = 100)

In [ ]:
# test.mean()

In [ ]:
# plt.subplots(figsize = (20,7))
# df.groupby(by='day').count()['value'].hist(bins = 100)